In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, AveragePooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.


In [2]:
data = pd.read_csv('data.csv')
print(data.shape)
dataset = np.array(data)
np.random.shuffle(dataset)

(72000, 1025)


In [3]:
x = dataset
y = dataset

x = x[:,0:1024]
y = y[:,1024]

x_train = x[0:70000,:]
x_train = x_train/255.
x_test = x[70000:72001,:]
x_test = x_test/255.

# Reshaping
y = y.reshape(y.shape[0],1)
y_train = y[0:70000,:]
y_train = y_train.T
y_test = y[70000:72001,:]
y_test = y_test.T


In [4]:
print('Number of training examples = '+str(x_train.shape[0]))
print('Number of test examples = '+str(x_test.shape[0]))
print('x_train shape ='+str(x_train.shape))
print('x_test shape ='+str(x_test.shape))
print('y_train shape = '+str(y_train.shape))
print('y_test shape = '+str(y_test.shape))

Number of training examples = 70000
Number of test examples = 2000
x_train shape =(70000, 1024)
x_test shape =(2000, 1024)
y_train shape = (1, 70000)
y_test shape = (1, 2000)


In [5]:
image_x = 32
image_y = 32

train_y = np_utils.to_categorical(y_train)
test_y = np_utils.to_categorical(y_test)
train_y = train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1],test_y.shape[2])

x_train = x_train.reshape(x_train.shape[0], image_x, image_y, 1)
x_test = x_test.reshape(x_test.shape[0], image_x, image_y, 1)

In [6]:
print('x_train shape:'+str(x_train.shape))
print('y_train shape:'+str(train_y.shape))

x_train shape:(70000, 32, 32, 1)
y_train shape:(70000, 37)


In [7]:
# Building Model.
def keras_model(image_x, image_y):
    num_of_classes = 37
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size=(5,5),
                    input_shape =(image_x,image_y,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))
    model.add(Conv2D(64, (5,5),activation='relu'))
    model.add(Flatten())
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = 'hindi_character_recognition_model.h5'
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint1]
    
    return model,callbacks_list

In [8]:
#Training the model.
model, callbacks_list = keras_model(image_x, image_y)
print(model)
model.fit(x_train, train_y , validation_data =(x_test, test_y), epochs=1 , batch_size=64 , callbacks=callbacks_list)
scores = model.evaluate(x_test, test_y, verbose=0)
print('CNN Error : %.2f%%'%(100-scores[1]*100))
print_summary(model)
model.save('hindi_character_recognition_model.h5')

Train on 70000 samples, validate on 2000 samples
Epoch 1/1
70000/70000 [==============================] - 149s 2ms/step - loss: 0.4677 - accuracy: 0.8666 - val_loss: 0.1625 - val_accuracy: 0.9485


C:\Users\jigyasa\Documents\satyam anaconda\envs\tensorflow\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


CNN Error : 5.15%
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 37)                236837    
Total params: 288,933
Trainable params: 288,933
Non-trainable params: 0
_________________________________________________________________
